In [1]:
# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import DataLoader

# OGB
from ogb.graphproppred import PygGraphPropPredDataset

# Utils
import tqdm

In [2]:
# useful stuff
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb#scrollTo=HvhgQoO8Svw4
# https://ogb.stanford.edu/docs/graphprop/#ogbg-mol

### Data loading

In [3]:
BATCH_SIZE = 32

dataset = PygGraphPropPredDataset(name = "ogbg-molhiv")

split_idx = dataset.get_idx_split() 
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

c:\Users\janbo\.conda\envs\future_financial_independence\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
# Example of molecule graph and target label
print(dataset[0])
print(dataset.num_node_features)
print(dataset.y)
print(f"Number of graphs: {len(dataset)}")
print(f"Class balance: {dataset.y.sum()}")

Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)
9
tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])
Number of graphs: 41127
Class balance: 1443


### GCN model

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, dataset.num_classes )

    def forward(self, x, edge_index, batch):
        # Conv layers
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # Readout layer
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return x

In [6]:
def train(model, optimizer, loss_fun):
    for data in train_loader:
        out = model(data.x.float(), data.edge_index, data.batch)
        loss = loss_fun(out, torch.reshape(data.y, (-1,)))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [7]:
def validate(loader, model):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x.float(), data.edge_index, data.batch)  
        pred = out.argmax(dim=1)
        correct += int((pred == torch.reshape(data.y, (-1,))).sum())
    acc = correct / len(loader.dataset)
    return acc

In [8]:
model = GCN(4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fun = nn.CrossEntropyLoss()

model.train()
for epoch in tqdm.tqdm(range(20)):
    train(model, optimizer, loss_fun)

print(f"Epoch: {epoch}, Train ACC: {validate(train_loader, model)}, Val ACC: {validate(valid_loader, model)}")
    

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [03:12<00:00,  9.61s/it]


Epoch: 19, Train ACC: 0.9625543296556336, Val ACC: 0.9803063457330415


In [9]:
print(f"Test ACC: {validate(test_loader, model)}")

Test ACC: 0.9683929005592026


In [10]:
# tutaj jest też metryka do tego datasetu (lub po prostu ROC-AUC)
# https://ogb.stanford.edu/docs/graphprop/#ogbg-mol

In [18]:
# calculating AUC 

from sklearn.metrics import roc_auc_score


def calc_auc(loader, model):
    model.eval()
    y_true = []
    y_pred = []
    for data in loader:
        out = model(data.x.float(), data.edge_index, data.batch)  
        
        y_true.append(data.y)
        
        # softmax to get probabilities
        y_pred.append(F.softmax(out, dim=1)[:, 1])

    return roc_auc_score(torch.cat(y_true).detach().numpy(), torch.cat(y_pred).detach().numpy())

In [19]:
calc_auc(train_loader,model)





0.5952201582480798

In [20]:
## Train parmeter 
hidden_channels = [2,8,16,32,64,128,256]

for hidden_channel in hidden_channels:
    model = GCN(hidden_channel)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fun = nn.CrossEntropyLoss()

    model.train()
    for epoch in tqdm.tqdm(range(20)):
        train(model, optimizer, loss_fun)

    print(f"hidden_channel: {hidden_channel}")
    print(f"Epoch: {epoch}, Train AUC: {calc_auc(train_loader, model)}, Val AUC: {calc_auc(valid_loader, model)}")
    #print(f"Test ACC: {validate(test_loader, model)}")
    #print(f"Test AUC: {calc_auc(test_loader,model)}")

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [02:58<00:00,  8.94s/it]


hidden_channel: 2
Epoch: 19, Train AUC: 0.5719013364907221, Val AUC: 0.6076098006074858


100%|██████████| 20/20 [03:11<00:00,  9.58s/it]


hidden_channel: 8
Epoch: 19, Train AUC: 0.5812376743531816, Val AUC: 0.5945415074466


 10%|█         | 2/20 [04:38<34:54, 116.37s/it]  

array([[ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.],
       [ 0., -1.]], dtype=float32)